# RL -- Projet "Trading automatique"

Ce notebook contient du code de base et quelques explications pour vous aider sur ce sujet.

Vous êtes libres de réaliser ce projet avec des scripts Python ou des Jupyter Notebooks, à votre convenance.

Vous devez télécharger les paquets Python suivants :

```sh
pip install gymnasium
pip install pandas
pip install gym-trading-env-continuous
```

Vous utiliserez l'environnement `gym-trading-env-continuous`, qui est un *fork* de [Gym Trading Env](https://gym-trading-env.readthedocs.io/en/latest/index.html). La différence majeure est expliquée dans ce document ; la documentation originelle reste utilisable.

In [4]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env

## Utilisation des données de simulation

Les données sont dans un format binaire (Pickle) que vous pouvez lire avec Pandas. Vous devez vous assurer que les données sont triées par date.

Des étapes de prétraitement peuvent aider votre apprentissage, par exemple, supprimer les doublons, etc.

In [5]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

df = preprocess(pd.read_pickle('./data/binance-ETHUSD-1h.pkl'))
df.head(5)

,open,high,low,close,volume,date_close
date_open,,,,,,
2020-08-18 07:00:00,430.00,435.00,410.00,430.30,487.154463,2020-08-18 08:00:00
2020-08-18 08:00:00,430.27,431.79,430.27,430.80,454.176153,2020-08-18 09:00:00
2020-08-18 09:00:00,430.86,431.13,428.71,429.35,1183.710884,2020-08-18 10:00:00
2020-08-18 10:00:00,429.75,432.69,428.59,431.90,1686.183227,2020-08-18 11:00:00
2020-08-18 11:00:00,432.09,432.89,426.99,427.45,1980.692724,2020-08-18 12:00:00


### Ajout de *features*

Vous pouvez également rajouter de nouvelles données au DataFrame pour créer de nouvelles *features* que l'agent pourra utiliser.
Voir pour cela la [doc](https://gym-trading-env.readthedocs.io/en/latest/features.html).

Chaque nouvelle *feature* doit commencer par `feature_` pour être détectée.

In [6]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()

    df['feature_close'] = (df['close'] - df['close'].mean()) / df['close'].std()

    return df

df = preprocess(pd.read_pickle('./data/binance-ETHUSD-1h.pkl'))
df.head(5)

,open,high,low,close,volume,date_close,feature_close
date_open,,,,,,,
2020-08-18 07:00:00,430.00,435.00,410.00,430.30,487.154463,2020-08-18 08:00:00,-1.891634
2020-08-18 08:00:00,430.27,431.79,430.27,430.80,454.176153,2020-08-18 09:00:00,-1.891128
2020-08-18 09:00:00,430.86,431.13,428.71,429.35,1183.710884,2020-08-18 10:00:00,-1.892594
2020-08-18 10:00:00,429.75,432.69,428.59,431.90,1686.183227,2020-08-18 11:00:00,-1.890016
2020-08-18 11:00:00,432.09,432.89,426.99,427.45,1980.692724,2020-08-18 12:00:00,-1.894514


Par défaut, l'agent ne reçoit comme *features* que sa dernière *position* (voir le paragraphe suivant), ce qui ne sera certainement pas suffisant ! À vous d'ajouter les *features* qui seront pertinentes pour que l'agent apprenne la politique optimale...

## Fonctionnement des actions

Une action est une **position**, c'est-à-dire un ratio entre la proportion d'*assets* (exemple : ETH) et la proportion de *fiat* (exemple : USD) dans le portefeuille.
Ainsi, la position `0.5` consiste à avoir exactement 50% d'ETH et 50% d'USD (en vendant l'un ou l'autre pour arriver à ce ratio). `0.1` consiste à avoir 10% d'ETH et 90% d'USD.

Il existe des positions un peu plus complexes :

- `< 0` : une position inférieure à 0 va vendre encore plus d'ETH que le portefeuille n'en contient, pour obtenir des USD. Cela nécessite un emprunt, qui sera remboursé avec un intérêt.
- `> 1` : une position supérieure à 1 va dépenser encore plus d'USD que le portefeuille n'en contient, pour acheter des ETH. Cela nécessite également un emprunt.

Ces positions (qui sont appelées *short* et *margin* en finance) peuvent faire gagner beaucoup à votre agent, mais démultiplient les risques également. Si votre agent fait une bonne affaire, vous pouvez vendre à un prix élevé, racheter quand le prix est plus faible, et rembourser l'emprunt en empochant la différence. En revanche, si votre agent fait une mauvaise affaire, et doit vider son portefeuille pour rembourser l'emprunt, vous perdez automatiquement (`terminated=True`).

### Actions continues

Par rapport à l'environnement `gym-trading-env` d'origine, la version que je vous fournis permet de spécifier directement une position comme action, c'est-à-dire un nombre flottant. Votre agent a donc un contrôle précis sur la position désirée. Cela rajoute de la flexibilité mais rend l'apprentissage beaucoup plus difficile.

Exemple :

In [7]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

obs, _ = env.reset()
# On veut une position de 88% ETH / 12% USD
obs, reward, terminated, truncated, info = env.step(0.88)
print(obs)
print(info)

[-0.9692127   0.88        0.87977463]
{'idx': 1, 'step': 1, 'date': np.datetime64('2023-11-21T15:30:00.000000000'), 'position_index': None, 'position': 0.88, 'real_position': np.float64(0.8797746150018296), 'data_open': 190.35000610351562, 'data_low': 189.74000549316406, 'data_high': 190.47000122070312, 'data_close': 189.94200134277344, 'data_volume': 5681421, 'data_date_close': Timestamp('2023-11-21 16:30:00'), 'portfolio_valuation': np.float64(997.359276756731), 'portfolio_distribution_asset': np.float64(4.6195752783697825), 'portfolio_distribution_fiat': np.float64(119.90790302957475), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': np.float64(-0.002644216103365779)}


Par défaut, l'espace des actions est limité à $[-1, 2]$ pour que votre agent ne puisse emprunter que jusqu'à 100%. Vous pouvez empêcher votre agent de prendre de telles positions, ou limiter le risque, en contrôlant les bornes autorisées des actions.

Par exemple, en clippant l'action dans l'intervalle $[0,1]$, vous empêchez l'agent de faire des emprunts.

À l'inverse, vous pouvez augmenter l'intervalle pour permettre des emprunts plus risqués, mais qui peuvent rapporter plus. À vous de choisir !

Vous pouvez changer les bornes via le paramètre `position_range` du constructeur :

In [8]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    position_range=(0, 1),  # ICI : (borne min, borne max)
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

Vous pouvez aussi modifier l'action en sortie de votre algorithme d'apprentissage, de la manière que vous souhaitez (clipping, interpolation, etc.).

### Actions discrètes

Pour simplifier l'apprentissage, vous pouvez utiliser le *wrapper* `gym_trading_env.wrapper.DiscreteActionsWrapper` que je vous fournis, et qui permet de revenir au fonctionnement d'origine de l'environnement `gym-trading-env`. Vous devrez alors spécifier l'ensemble des positions possibles, puis votre agent choisira une position parmi cette liste à chaque pas de temps.
Par exemple, si la liste des positions est `[0, 0.5, 1]` et que l'action choisie est `1`, cela veut dire qu'on veut la position qui correspond au 2e élément de la liste, soit `0.5` (50%/50%).

Vous pouvez rajouter autant d'actions que vous voulez, par exemple `[0, 0.25, 0.5, 1]` ou encore tous les 0.1 entre 0 et 1, etc. Plus il y a d'actions possibles, plus votre agent aura de choix (flexibilité), donc plus son comportement pourra être complexe, mais cela rajoute de la difficulté durant l'entraînement.

N'oubliez pas que vous pouvez autoriser les positions avec emprunt en ajoutant des nombres inférieurs à 0 ou supérieurs à 1 à la liste autorisée.

Exemple :

In [9]:
from gym_trading_env.wrapper import DiscreteActionsWrapper

# Vous pouvez aussi appeler le wrapper `env` pour faire plus simple
# Ici, je fais explicitement la distinction entre `wrapper` et `env`
wrapper = DiscreteActionsWrapper(env, positions=[-1, 0, 0.25, 0.5, 0.75, 1, 2])
obs, _ = wrapper.reset()
# On veut une position de 25% ETH / 75% USD ; cela correspond à la position
# d'index 2 dans la liste ci-dessus
obs, reward, terminated, truncated, info = wrapper.step(2)
print(obs)
print(info)

[-1.691366    0.25        0.25039366]
{'idx': 1, 'step': 1, 'date': np.datetime64('2023-11-21T14:00:00.000000000'), 'position_index': 2, 'position': 0.25, 'real_position': np.float64(0.2503936737643807), 'data_open': 7216.89990234375, 'data_low': 7216.89990234375, 'data_high': 7234.830078125, 'data_close': 7231.89013671875, 'data_volume': 0, 'data_date_close': Timestamp('2023-11-21 15:00:00'), 'portfolio_valuation': np.float64(1000.1504441153436), 'portfolio_distribution_asset': np.float64(0.03462875393358), 'portfolio_distribution_fiat': np.float64(749.7191000962257), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': np.float64(0.00015043279976245777)}


Notez que, quand les actions continues sont utilisées, la variable `position_index` du dictionnaire `info` n'est pas disponible (c'est logique).

## Changement de la fonction de récompense

Vous pouvez changer la fonction de récompense pour améliorer l'apprentissage de l'agent.
Dans tous les cas, vous serez évalué(e)s sur la valuation du portefeuille à la fin de l'épisode (voir [ci-dessous](#évaluation)), mais cette simple mesure n'est peut-être pas la meilleure fonction de récompense.
D'autres fonctions peuvent encourager l'agent à mieux apprendre, en explorant diverses possibilités, etc.

In [10]:
def reward_function(history):
    return history['portfolio_valuation', -1]

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # On spécifie la fonction de récompense
    reward_function=reward_function,
)

## Déroulément d'un épisode

Un épisode se déroule jusqu'à ce que :

- l'agent atteigne la fin des données d'entraînement (nous n'avons plus de nouvelle donnée) => `truncated=True`

- la valeur du portefeuille atteint 0 (l'agent a perdu tout l'argent) => `terminated=True`

Vous devrez probablement entraîner l'agent sur plusieurs épisodes avant que son comportement ne converge.

Pour éviter de sur-apprendre (*overfit*), vous devrez utiliser plusieurs jeux de données via [MultiDatasetTradingEnv](https://gym-trading-env.readthedocs.io/en/latest/multi_datasets.html).

Dans ce cas, chaque épisode utilisera un jeu de données différent (en bouclant si vous demandez plus d'épisodes qu'il n'y a de jeux de données). Vous pouvez accéder au nom du jeu de données de l'épisode en cours via `env.name`.

In [11]:
nb_episodes = 2
for episode in range(1, nb_episodes + 1):
    obs, _ = env.reset()
    print(f'Episode n˚{episode} -- Jeu de donnée {env.name}')
    done = False

    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    if terminated:
        print('Argent perdu')
    elif truncated:
        print('Épisode terminé')


Episode n˚1 -- Jeu de donnée binance-ETHUSD-1h.pkl
Market Return : 904.05%   |   Portfolio Return : -100.00%   |   
Épisode terminé
Episode n˚2 -- Jeu de donnée yfinance-STOXX50-1h.pkl
Market Return : 27.70%   |   Portfolio Return : -98.47%   |   
Épisode terminé


## Évaluation

Afin de disposer d'un critère simple pour comparer les différentes solutions, nous utiliserons la valeur du portefeuille (`portfolio_valuation`).
C'est assez simple : on veut que l'agent ait gagné le plus d'argent à la fin de la simulation.

Vous pouvez ajouter ce critère à la liste des métriques affichées à la fin de chaque épisode, pour que ce soit plus visible :

In [12]:
def metric_portfolio_valuation(history):
    return round(history['portfolio_valuation', -1], 2)

env.add_metric('Portfolio Valuation', metric_portfolio_valuation)

done = False
obs, _ = env.reset()

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

Market Return : 163.91%   |   Portfolio Return : -100.00%   |   Portfolio Valuation : 0.0   |   


Puisque l'environnement peut se dérouler sur plusieurs épisodes (1 par jeu de données), vous devrez calculer la **moyenne des `portfolio_valuation`** sur l'ensemble des jeux de données possibles.

⚠️ Pour que ce soit honnête, vous **devez initialiser l'environnement avec les contraintes** imposées dans le sujet :

- une valeur initiale du portefeuille de `1000` ;
- des frais de 0.1% par transaction ;
- un taux d'intérêt de 0.02% par jour soit 0.02/100/24 par heure.

Sinon, il est beaucoup plus simple d'augmenter la valeur finale...

```py
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    # LIGNES SUIVANTES :
    # Valeur initiale du portefeuille
    portfolio_initial_value=1_000,
    # Frais de transactions
    trading_fees=0.1/100,
    # Intérêts sur les prêts
    borrow_interest_rate=0.02/100/24,
)
```

Vous pouvez également accéder à la métrique de `portfolio_valuation` à la fin d'une simulation, si vous voulez par exemple l'ajouter à votre *run* WandB :

In [13]:
portfolio_valuation = env.historical_info['portfolio_valuation', -1]
# Si on avait WandB :
# run.summary['portfolio_valuation'] = portfolio_valuation
# On simule ça par un simple print...
print(portfolio_valuation)

9.929165601714467e-18


Ou bien, pour récupérer les métriques calculées par l'environnement (cela peut être utile pour les ajouter à WandB) :

In [14]:
metrics = env.get_metrics()
print(metrics)
portfolio_valuation = metrics['Portfolio Valuation']
print(portfolio_valuation)

{'Market Return': '163.91%', 'Portfolio Return': '-100.00%', 'Portfolio Valuation': np.float64(0.0)}
0.0


## Conseils

À part les quelques contraintes présentées dans ce fichier (et rappelées sur la page du projet), vous êtes assez libres !

Votre algorithme de RL peut être arbitrairement simple ou complexe. Je liste ici quelques conseils ou pistes, que vous pouvez explorer :

- *Features* : Par défaut, votre agent n'utilise que le prix de l'*asset* (`close`) comme *feature* pour la prise de décision. Vous pouvez ajouter les *features* que vous voulez. En particulier, des métriques spécifiques à la finance peuvent être intéressantes, par exemple pour déterminer le risque que le prix change brutalement (à la hausse ou à la baisse)...

- Algorithme : Vous pouvez utiliser des algorithmes existants, ou en inventer un nouveau. N'hésitez pas à ré-utiliser tout ce que vous avez appris en *Machine Learning* et *Deep Learning*. Typiquement, les données financières sont des données temporelles : certains réseaux de neurones sont plus appropriés que d'autres pour ce genre de tâche...

- Configuration de l'environnement : L'environnement est très extensible ! Vous pouvez par exemple ajouter des *features* dynamiques (pas seulement calculées lors du prétraitement). La [documentation](https://gym-trading-env.readthedocs.io/en/latest/index.html) est très claire et très complète.

Vous pouvez vous inspirer de travaux existants trouvés sur l'Internet à condition de **citer votre source**. Utiliser le travail de quelqu'un d'autre sans le citer sera considéré comme du plagiat.

# Version aléatoire complète avec RSI et MACD, et visualisation

In [13]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env
from gym_trading_env.wrapper import DiscreteActionsWrapper
import os

## 1. Définition des indicateurs techniques

In [14]:
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(series, slow=26, fast=12, signal=9):
    exp1 = series.ewm(span=fast, adjust=False).mean()
    exp2 = series.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    return macd

## 2. Prétraitement des données

In [15]:
def preprocess(df):
    # Tri et nettoyage
    df = df.sort_index().dropna().drop_duplicates()

    # Ajout de features (doivent commencer par "feature_")
    # 1. RSI normalisé entre 0 et 1
    df['feature_RSI'] = calculate_rsi(df['close']) / 100

    # 2. MACD
    df['feature_MACD'] = calculate_macd(df['close'])

    # 3. Rendements logarithmiques (plus stable pour le RL que le prix brut)
    df['feature_log_return'] = np.log(df['close'] / df['close'].shift(1))

    # 4. Position du prix par rapport à la moyenne mobile
    df['feature_sma_dist'] = (df['close'] - df['close'].rolling(20).mean()) / df['close'].rolling(20).std()

    return df.dropna()

## 3. Configuration de l'environnement

In [16]:
def reward_function(history):
    # Récompense basée sur la variation logarithmique de la valeur du portefeuille
    # Cela encourage une croissance stable plutôt que des paris risqués
    if len(history["portfolio_valuation"]) < 2:
        return 0
    return np.log(history['portfolio_valuation', -1] / history['portfolio_valuation', -2])

In [17]:
# Création de l'environnement avec les contraintes du projet
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="./data/*.pkl", # Assure-toi que le dossier data contient tes fichiers .pkl
    preprocess=preprocess,
    portfolio_initial_value=1000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    reward_function=reward_function,
)

## 4. Exécution d'une simulation (Exemple avec un agent aléatoire)

In [19]:
print(f"Démarrage de la simulation sur le dataset : {env.unwrapped.name}")

obs, info = env.reset()
done = False
truncated = False

while not (done or truncated):
    # Ici, tu remplaceras par : action, _states = model.predict(obs) si tu utilises Stable Baselines
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)

Démarrage de la simulation sur le dataset : binance-ETHUSD-1h.pkl
Market Return : 26.85%   |   Portfolio Return : -98.80%   |   


## 5. Visualisation

In [19]:
from gym_trading_env.renderer import Renderer

print("Sauvegarde des logs pour la visualisation...")
env.unwrapped.save_for_render(dir="render_logs")

print("Simulation terminée.")
print(f"Valeur finale du portefeuille : {info['portfolio_valuation']:.2f}$")

renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

Sauvegarde des logs pour la visualisation...
Simulation terminée.
Valeur finale du portefeuille : 0.00$
 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


# Version avec Stable Baselines3 et PPO

## 1. Création de l'environnement d'entraînement

In [20]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="./data/*.pkl",
    preprocess=preprocess, # Ta fonction avec RSI, MACD, etc.
    portfolio_initial_value=1000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    reward_function=reward_function,
)

## 2. Création de l'agent PPO avec MLP

In [21]:
from stable_baselines3 import PPO

model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.0003, # Hyperparamètre à ajuster
    gamma=0.99,           # Facteur de réduction
    tensorboard_log="./ppo_tensorboard/"
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## 3. Apprentissage

In [22]:
print("Entraînement en cours...")
model.learn(total_timesteps=100000) # Augmente ce chiffre pour de meilleures performances
model.save("mon_agent_trading")

Entraînement en cours...
Logging to ./ppo_tensorboard/PPO_2
-----------------------------
| time/              |      |
|    fps             | 2123 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1229         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0046373173 |
|    clip_fraction        | 0.026        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.41        |
|    explained_variance   | -2.24        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.021        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00161     |
|    std                  | 0.994        |

 ## 4. Évaluation et Visualisation sur un épisode

In [23]:
print("Évaluation de l'agent entraîné...")
obs, info = env.reset()
done, truncated = False, False

while not (done or truncated):
    # L'agent utilise maintenant son expérience pour choisir l'action
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)

Évaluation de l'agent entraîné...
Market Return : 39.93%   |   Portfolio Return : -38.29%   |   


## 5. Visualisation

In [24]:
env.unwrapped.save_for_render(dir="render_logs")
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Dec/2025 16:13:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:13:37] "GET /update_data/yfinance-GOLDUSD-1h.pkl_2025-12-17_14-21-49.pkl HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:13:38] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:13:44] "GET /update_data/yfinance-GOLDUSD-1h.pkl_2025-12-17_14-13-08.pkl HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:13:44] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:14:20] "GET /update_data/binance-DOGEEUR-1h.pkl_2025-12-17_14-13-11.pkl HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:14:20] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:15:59] "GET /update_data/yfinance-GOLDUSD-1h.pkl_2025-12-17_14-13-08.pkl HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 16:15:59] "GET /metrics HTTP/1.1" 200 -


In [44]:
df= pd.read_pickle('./render_logs/yfinance-GOLDUSD-1h.pkl_2025-12-17_14-21-49.pkl')
df.head()
df.columns
df.position.unique()
df["position"] = df.position.astype(float)
df.to_pickle('./render_logs/yfinance-GOLDUSD-1h.pkl_2025-12-17_14-21-49.pkl')

# RecurrentPPO + Gestion du Risque

In [58]:
import gymnasium as gym
import gym_trading_env
from gym_trading_env.wrapper import DiscreteActionsWrapper
from stable_baselines3 import PPO
from gym_trading_env.renderer import Renderer
import numpy as np

# --- 1. Preprocess (On garde tes indicateurs) ---
def preprocess(df):
    df = df.sort_index().dropna().drop_duplicates()
    df['feature_close'] = df['close'].pct_change()
    df['feature_rsi'] = calculate_rsi(df['close']) / 100
    df['feature_macd'] = calculate_macd(df['close'])
    return df.dropna()

# --- 2. Nouvelle fonction de récompense stricte ---
def reward_function(history):
    # 1. Calcul du rendement réel du portefeuille (frais inclus par l'env)
    current_val = history['portfolio_valuation', -1]
    prev_val = history['portfolio_valuation', -2]

    # Rendement logarithmique
    reward = np.log(current_val / prev_val)

    # 2. PÉNALITÉ DE CHANGEMENT (Anti-Churning)
    # Si l'agent change de position, il paie des frais.
    # On ajoute une punition supplémentaire pour l'inciter à "Hold".
    current_pos = history['position', -1]
    prev_pos = history['position', -2]

    if current_pos != prev_pos:
        # On lui enlève artificiellement un peu plus de reward
        # pour qu'il ne trade que si c'est vraiment nécessaire
        reward -= 0.0005

    return reward

# --- 3. Création de l'environnement de base ---
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="./data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    reward_function=reward_function,
)

# --- 4. LE WRAPPER DISCRET (La solution miracle) ---
env = DiscreteActionsWrapper(env, positions=[-0.25, 0, 1, 0.25,  0.5, 0.75])

# --- 5. Agent PPO ---
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.0003,
    ent_coef=0.01, # Encourage l'exploration pour ne pas rester bloqué sur 0
    tensorboard_log="./ppo_discrete_tensorboard/"
)

print("Entraînement en mode 'Sécurité' (Actions Discrètes)...")
model.learn(total_timesteps=100_000)
model.save("ppo_discrete_safe")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Entraînement en mode 'Sécurité' (Actions Discrètes)...
Logging to ./ppo_discrete_tensorboard/PPO_11
-----------------------------
| time/              |      |
|    fps             | 2070 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1387        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011655919 |
|    clip_fraction        | 0.0992      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | -7.74       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0551     |
|    n_u

In [59]:
# --- 6. Visualisation ---
print("Lancement de la simulation...")
obs, info = env.reset()
done, truncated = False, False

while not (done or truncated):
    action, _ = model.predict(obs)

    # --- CORRECTION ---
    # On force la conversion du tableau numpy vers un entier Python standard
    action = int(action)
    # ------------------

    obs, reward, done, truncated, info = env.step(action)

# Sauvegarde
env.unwrapped.save_for_render(dir="render_logs")
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

Lancement de la simulation...
Market Return : 914.64%   |   Portfolio Return : -99.46%   |   
 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Dec/2025 17:30:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 17:30:55] "GET /update_data/yfinance-S&P500-1h.pkl_2025-12-17_17-27-29.pkl HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2025 17:30:55] "GET /metrics HTTP/1.1" 200 -


# Solution Hybride & Anti-Short Bias

In [1]:
import gymnasium as gym
import gym_trading_env
from gym_trading_env.wrapper import DiscreteActionsWrapper
from stable_baselines3 import PPO
from gym_trading_env.renderer import Renderer
import numpy as np
import pandas as pd
import wandb
from wandb.integration.sb3 import WandbCallback

# --- 1. CONFIGURATION ET INDICATEURS ---

# On définit les hyperparamètres ici pour que WandB puisse les enregistrer
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 200_000,
    "learning_rate": 0.0003,
    "ent_coef": 0.02, # Coefficient d'exploration
    "batch_size": 128,
    "positions": [-0.5, 0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5], # Hybride
    "project_name": "RL-Trading-Project"
}

def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(series, slow=26, fast=12, signal=9):
    exp1 = series.ewm(span=fast, adjust=False).mean()
    exp2 = series.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    return macd

def preprocess(df):
    df = df.sort_index().dropna().drop_duplicates()
    df['feature_close'] = df['close'].pct_change()
    df['feature_rsi'] = calculate_rsi(df['close']) / 100
    df['feature_macd'] = calculate_macd(df['close'])
    return df.dropna()

def reward_function(history):
    current_val = history['portfolio_valuation', -1]
    prev_val = history['portfolio_valuation', -2]
    reward = np.log(current_val / prev_val)

    # Malus pour les positions Short (pour éviter le biais négatif)
    if history['position', -1] < 0:
        reward -= 0.0002

    return reward

# --- 2. INITIALISATION DE WANDB ---
run = wandb.init(
    project=config["project_name"],
    config=config,
    sync_tensorboard=True, # Synchronise automatiquement les logs SB3
    monitor_gym=True,      # Essaie d'enregistrer les vidéos (si disponible)
    save_code=True,        # Sauvegarde ce script dans WandB
)

# --- 3. CRÉATION DE L'ENVIRONNEMENT ---
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="./data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    reward_function=reward_function,
)

# Wrapper Hybride (Int -> Float spécifique)
env = DiscreteActionsWrapper(env, positions=config["positions"])

# --- 4. ENTRAÎNEMENT AVEC CALLBACK WANDB ---
model = PPO(
    config["policy_type"],
    env,
    verbose=1,
    learning_rate=config["learning_rate"],
    ent_coef=config["ent_coef"],
    batch_size=config["batch_size"],
    tensorboard_log=f"runs/{run.id}" # Dossier unique pour Tensorboard
)

print(f"Lancement de l'entraînement WandB : {run.name}")
model.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"models/{run.id}",
        verbose=2,
    )
)
model.save("ppo_trading_wandb_final")

# --- 5. ÉVALUATION ET LOGGING FINAL ---
print("Évaluation finale...")
obs, info = env.reset()
done, truncated = False, False

while not (done or truncated):
    action, _ = model.predict(obs)
    action = int(action) # Conversion array -> int pour le wrapper
    obs, reward, done, truncated, info = env.step(action)

# Récupération des métriques finales de l'environnement
final_metrics = env.unwrapped.get_metrics()
print("Métriques finales :", final_metrics)

# Envoi des métriques clés à WandB (pour le tableau de bord)
wandb.log({
    "final_portfolio_valuation": info['portfolio_valuation'],
    "market_return": final_metrics.get("Market Return", 0),
    "portfolio_return": final_metrics.get("Portfolio Return", 0)
})

# --- 6. VISUALISATION ---
env.unwrapped.save_for_render(dir="render_logs")

# On ferme le run WandB proprement
wandb.finish()

wandb: Currently logged in as: arthur-collignon (arthur-collignon-cpe-lyon) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Lancement de l'entraînement WandB : firm-lake-9
Logging to runs/n6etkjbb\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1066 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 890         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009804827 |
|    clip_fraction        | 0.0478      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.07       |
|    explained_variance   | -1.94       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0623     |
|    n_updates            | 1

wandb: WARNING Linked 1 file into the W&B run directory (hardlinks); call wandb.save again to sync new files.


Évaluation finale...
Market Return :  9.60%   |   Portfolio Return : -64.89%   |   
Métriques finales : {'Market Return': ' 9.60%', 'Portfolio Return': '-64.89%'}


final_portfolio_valuation,▁
global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇████
rollout/ep_len_mean,██▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▂▂▂▂▂▁▁▁
rollout/ep_rew_mean,▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████
time/fps,█▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/approx_kl,▃▆▄▃▂▃▃▂▂▃▄▂█▁▂▁▁▃▃▄▅▄▂▄▃▂▂▂▃▃▂▃▂▃▂▂▂▂▂▂
train/clip_fraction,▂▄▄▄▄▁▂▅▃▃▅▅█▇▄▃▂▁▂▂▃▁▁▃▅▃▂▂▂▃▃▅▃▄▄▂▃▂▅▃
train/clip_range,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/entropy_loss,▁▁▁▁▁▂▂▂▂▂▂▄█▇▇▆▆▅▅▅▄▄▄▄▇▆▇▇▆▅▆▆▅▆▅▇█▇▃▃
train/explained_variance,▁███▆█▇███▇▇▇▅▁█████████▆█▇██████▇██████
+4,...


In [2]:
# Lancement du renderer local
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2025 10:27:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 10:27:50] "GET /update_data/yfinance-S&P500-1h.pkl_2025-12-17_17-42-49.pkl HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2025 10:27:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2025 10:27:51] "GET /metrics HTTP/1.1" 200 -


# Agent : RecurentPPO

In [15]:
import gymnasium as gym
import gym_trading_env
from gym_trading_env.wrapper import DiscreteActionsWrapper
from gym_trading_env.renderer import Renderer
import numpy as np
import pandas as pd
import wandb
from wandb.integration.sb3 import WandbCallback

# Vous aurez besoin de cette librairie pour le LSTM
# !pip install sb3-contrib

from sb3_contrib import RecurrentPPO

# --- 1. CONFIGURATION (WandB) ---
config = {
    "policy_type": "MlpLstmPolicy",  # Changement majeur : LSTM
    "total_timesteps": 500_000,      # Un peu plus long pour le LSTM
    "learning_rate": 3e-4,
    "ent_coef": 0.01,
    "batch_size": 128,
    "n_steps": 2048,
    "window_size": 20,               # Fenêtre d'observation pour le LSTM
    "positions": [0, 0.5, 1.0],      # Simplifié au début : Cash, Moitié, Full (Pas de levier/short risqué)
    "project_name": "RL-Trading-Project",
    "run_name": "RecurrentPPO_Optimized"
}

# --- 2. FONCTIONS DE TRAITEMENT ---

def calculate_indicators(df):
    # RSI
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['feature_rsi'] = 100 - (100 / (1 + rs))
    df['feature_rsi'] = df['feature_rsi'] / 100.0 # Normalisé

    # MACD
    exp1 = df['close'].ewm(span=12, adjust=False).mean()
    exp2 = df['close'].ewm(span=26, adjust=False).mean()
    df['feature_macd'] = (exp1 - exp2) / df['close'] # Normalisé par le prix

    # ATR (Volatilité) - Important pour la survie
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['feature_atr'] = true_range.rolling(14).mean() / df['close']

    # Returns
    df['feature_return'] = df['close'].pct_change()

    return df.dropna()

def preprocess(df):
    df = df.sort_index().dropna().drop_duplicates()
    return calculate_indicators(df)

def reward_function(history):
    # Rendement logarithmique
    current_val = history['portfolio_valuation', -1]
    prev_val = history['portfolio_valuation', -2]
    ret = np.log(current_val / prev_val)

    # Pénalité de volatilité (Sharpe Ratio implicite)
    # Cela calme l'agent pour éviter les -50%
    risk_penalty = 0.1 * (ret ** 2)

    return ret - risk_penalty

# --- 3. INITIALISATION WANDB ---
run = wandb.init(
    project=config["project_name"],
    name=config["run_name"],
    config=config,
    sync_tensorboard=True, # Synchronise les logs SB3 avec WandB
    monitor_gym=True,
    save_code=True,
)

# --- 4. CRÉATION DE L'ENVIRONNEMENT ---
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="./data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    reward_function=reward_function,
    window_size=config["window_size"] # Important pour le LSTM
)

env = DiscreteActionsWrapper(env, positions=config["positions"])

# --- 5. CRÉATION DU MODÈLE ET ENTRAÎNEMENT ---
model = RecurrentPPO(
    config["policy_type"],
    env,
    verbose=1,
    learning_rate=config["learning_rate"],
    ent_coef=config["ent_coef"],
    batch_size=config["batch_size"],
    n_steps=config["n_steps"],
    # Log dans le dossier spécifique pour que WandB le trouve
    tensorboard_log=f"runs/{run.id}"
)

print(f"Lancement du run WandB : {run.name}")
model.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"models/{run.id}",
        verbose=2,
    )
)

model.save("recurrent_ppo_final")

# --- 6. ÉVALUATION ET LOGGING FINAL ---
print("Évaluation...")
obs, info = env.reset()
done, truncated = False, False

while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    action = int(action)
    obs, reward, done, truncated, info = env.step(action)

# Envoi des métriques finales manuelles
final_metrics = env.unwrapped.get_metrics()
wandb.log({
    "final_portfolio_valuation": info['portfolio_valuation'],
    "market_return": final_metrics.get("Market Return", 0),
    "portfolio_return": final_metrics.get("Portfolio Return", 0)
})

# Fin du run
wandb.finish()

TypeError: TradingEnv.__init__() got an unexpected keyword argument 'window_size' was raised from the environment creator for MultiDatasetTradingEnv with kwargs ({'dataset_dir': './data/*.pkl', 'preprocess': <function preprocess at 0x000001BBC3912200>, 'portfolio_initial_value': 1000, 'trading_fees': 0.001, 'borrow_interest_rate': 8.333333333333334e-06, 'reward_function': <function reward_function at 0x000001BBC39132E0>, 'window_size': 20})

In [ ]:
# Visualisation locale
env.unwrapped.save_for_render(dir="render_logs")
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()